In [ ]:
# for find out more details if needed
%set_env LLAMA_LOG_VERBOSITY=4

# Quick build

In [ ]:
#build for cpu debug
%cd llama.cpp
!cmake -B cpu_debug -DGGML_CUDA=OFF -DGGML_RPC=OFF -DGGML_BLAS=OFF \
    -DGGML_SCHED_MAX_COPIES=1 -DLLAMA_CURL=OFF -DGGML_CPU_REPACK=OFF -DCMAKE_BUILD_TYPE=Debug
!cmake --build cpu_debug --config Debug -j $(nproc)
%cd ..

In [ ]:
#build for gpu debug
%cd llama.cpp
!cmake -B cuda_debug -DGGML_CUDA=ON -DGGML_RPC=OFF -DGGML_BLAS=OFF \
    -DGGML_SCHED_MAX_COPIES=1 -DLLAMA_CURL=OFF -DGGML_CPU_REPACK=OFF -DCMAKE_BUILD_TYPE=Debug
!cmake --build cuda_debug --config Debug -j $(nproc)
%cd ..

In [ ]:
#build for cpu execution
%cd llama.cpp
!cmake -B cpu_build -DGGML_CUDA=OFF -DGGML_RPC=OFF -DGGML_BLAS=OFF \
    -DGGML_SCHED_MAX_COPIES=1 -DLLAMA_CURL=OFF -DGGML_CPU_REPACK=OFF
!cmake --build cpu_build --config Release -j $(nproc)
%cd ..

In [ ]:
#build for gpu execution
%cd llama.cpp
!cmake -B cuda_build -DGGML_CUDA=ON -DGGML_RPC=OFF -DGGML_BLAS=OFF \
    -DGGML_SCHED_MAX_COPIES=1 -DLLAMA_CURL=OFF -DGGML_CPU_REPACK=OFF
!cmake --build cuda_build --config Release -j $(nproc)
%cd ..

In [ ]:
!git clone https://github.com/ikawrakow/ik_llama.cpp --depth 1
%cd ik_llama.cpp
!cmake -B cuda_build -DGGML_CUDA=ON -DGGML_RPC=OFF -DGGML_BLAS=OFF -DGGML_SCHED_MAX_COPIES=1 -DLLAMA_CURL=OFF
!cmake --build cuda_build --config Release -j $(nproc)
%cd ..

# 42 test

- [TinyStories-656K](https://hf-mirror.com/mradermacher/TinyStories-656K-GGUF)

<details>
<summary>config</summary>

```json
{
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 384,
  "max_position_embeddings": 512,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 8,
  "num_hidden_layers": 2,
  "num_key_value_heads": 4,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.41.2",
  "use_cache": true,
  "vocab_size": 2048
}
```
</details>

In [ ]:
model = "models/TinyStories-656K.f16.gguf"
assert os.path.exists(model)
!llama.cpp/cpu_build/bin/llama-cli -m {model} -p "the answer to the ultimate question of life, the universe, and everything is 42. One day, they " \
-t 1 -n 2 --temp 0 --top-k 0 --top-p 1.0 --min-p 0.0 2>/dev/null

# Hello world test

- [SmolLM2-135M](https://hf-mirror.com/bartowski/SmolLM2-135M-Instruct-GGUF/tree/main)

<details>
<summary>Prompt format</summary>


```
<|im_start|>system
{system_prompt}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
```

</details>

In [ ]:
prefix = "<|im_start|>system\nYou are a helpful assistant<|im_end|>\n<|im_start|>user\n"
suffix = "<|im_end|>\n<|im_start|>assistant\n"
# Q4_K_L == Q4_K_M, Q5_K_L == Q5_K_M
model = "models/SmolLM2-135M-Instruct-f16.gguf"
assert os.path.exists(model)

In [ ]:
question = "Give the most simple C program to print 'hello world!'" # Q4_K_S or under will be failed
prompt = prefix + question + suffix
!llama.cpp/cpu_build/bin/llama-cli -m {model} -p "{prompt}" -no-cnv -t 1 --temp 0 --top-k 0 --top-p 1.0 --min-p 0.0 2>/dev/null

In [ ]:
question = "Give the most simple C program to print 'Love world!'" # not work for Q8_0/Q6_K, Q5_K_S is the lowest, Q4 or under will be failed
# question = "Give the most simple C program to print 'love world!'" # Q4 or under will be failed
prompt = prefix + question + suffix
!llama.cpp/cpu_build/bin/llama-cli -m {model} -p "{prompt}" -no-cnv -t 1 --temp 0 --top-k 0 --top-p 1.0 --min-p 0.0 2>/dev/null

# Manga auther test

- [Llama-3.2-1B](https://hf-mirror.com/bartowski/Llama-3.2-1B-Instruct-GGUF/tree/main)
- [Official text_prompt_format](https://github.com/meta-llama/llama-models/blob/main/models/llama3_2/text_prompt_format.md)

<details>
<summary>Prompt format</summary>

```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>

{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
```


```python
# Q4_K_M/S 展现出了随着prefix和suffix微小变化的不稳定性
# prefix = "<|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nYou are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
# prefix = "<|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Dec 2024\n\nYou are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
# prefix = "<|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Oct 2024\n\nYou are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
# suffix = "<|eot_id|><|start_header_id|>assistant<|end_header_id|>" # Q4_K_M only corrent for 火影忍者的作者是谁？ pair with Jul 2024
```

</details>

In [ ]:
# Q4_0/awq will be failed
question = "Who is the author of 'Demon Slayer'?"
answer = "The author of the popular manga and anime series 'Demon Slayer: Kimetsu no Yaiba' is Koyoharu Gotoge. [end of text]"

In [ ]:
# awq will be failed
question = "Who is the author of 'Chainsaw Man'?"
answer = "The author of the manga and anime series 'Chainsaw Man' is Tatsuki Fujimoto. [end of text]"

In [ ]:
# [awq] will be failed
question = "Who is the author of 'Detective Conan'?"
answer = "The author of the popular manga and anime series 'Detective Conan' is Gosho Aoyama. [end of text]"

In [ ]:
# under-4-bit will be failed
question = "Who is the author of Manga 'Slam Dunk'?"
answer = 'The author of the popular manga series "Slam Dunk" is Takehiko Inoue. [end of text]'

In [ ]:
# under-4-bit will be failed
question = "Who is the author of 'Berserk'?"
answer = 'The author of the manga and anime series "Berserk" is Kentaro Miura. [end of text]'

In [ ]:
# **Q5_K_M/Q5_K_S**/[Q4_K_M/Q4_K_S]/IQ4_XS will be failed
question = "Naruto的作者是谁？"
answer = "Naruto的作者是Masashi Kishimoto [end of text]"

In [ ]:
# [Q4_K_M/Q4_K_S]/IQ4_XS/awq/gptq will be failed
question = "火影忍者的作者是谁？"
answer = "火影忍者是由Masashi Kishimoto所创作的日本动画和漫画。 [end of text]"

In [ ]:
prefix = "<|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Aug 2024\n\nYou are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
suffix = "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
prompt = prefix + question + suffix
!llama.cpp/cpu_build/bin/llama-cli -m {model} -p "{prompt}" -c 2048 -n 512 --temp 0 --top-k 1 --seed 42 -ngl 100 2>/dev/null
print("\n"+answer)
model

# Quantization

In [ ]:
# model_name = "Llama-3.2-1B-Instruct"
model_name = "Meta-Llama-3.1-8B-Instruct"
hf_model = f"models/{model_name}"

## gguf

In [ ]:
bf16_model = f"models/{model_name}-BF16.gguf"

In [ ]:
convert_hf_to_gguf_tool = f"llama.cpp/convert_hf_to_gguf.py"
!python {convert_hf_to_gguf_tool} --outtype bf16 {hf_model} --outfile {bf16_model}

In [ ]:
# Get the imatrix file
calib_file = f"calibration_data/calibration_datav3.txt"
imatrix_file = f"models/{model_name}.imatrix"
# to use cuda backend, otherwise will be very very slow
!ik_llama.cpp/cuda_build/bin/llama-imatrix -m {bf16_model} -f {calib_file} --output-file {imatrix_file} -ngl 100

In [ ]:
import os
quant_type = "IQ2_K"
# quant_type = "IQ2_KL"
# quant_type = "IQ2_S"
# quant_type = "IQ2_M"
# quant_type = "IQ2_KT"
# quant_type = "exl3-2.0bpw"
# quant_type = "exl3-2.25bpw"
# quant_type = "exl3-2.5bpw"

# quant_type = "Q4_0"
# quant_type = "IQ4_XS"
# quant_type = "IQ3_XXS"
# quant_type = "IQ3_KT"
# quant_type = "exl3-3.0bpw"

# quant_type += "-IQ2_KS"
# quant_type += "-dequant"

model = f"models/{model_name}-{quant_type}.gguf"
assert os.path.exists(model)
model

In [ ]:
# Get the quant model
imatrix_file = f"--imatrix models/{model_name}.imatrix"
# imatrix_file = ""

quant_option = "--token-embedding-type q6_K --output-tensor-type q4_K"
# quant_option += " --pure"

# use cpu_build is ok because only cpu is needed
!ik_llama.cpp/cuda_build/bin/llama-quantize {quant_option} {imatrix_file} {bf16_model} {model} {quant_type}
model

In [ ]:
# use gguf_dump.py to check quant_model
# the gguf_dump.py file is in (ik_)llama.cpp/scripts folder
gguf_dump_tool = "llama.cpp/gguf-py/gguf/scripts/gguf_dump.py"
!python {gguf_dump_tool} --markdown {model}

<details>
<summary>perplexity results</summary>

Attention: use ik_llama.cpp for PPL testing with `--output-tensor-type q6_K --token-embedding-type q6_K`

```bash
# wc -c < models/Llama-3.2-1B-Instruct-Q2_K.gguf  | awk '{printf "%.2f bpw\n", ($1/1024/1024-7.5-205.49)/1856.2*16}'
# wc -c < models/Llama-3.2-3B-Instruct-Q2_K.gguf  | awk '{printf "%.2f bpw\n", ($1/1024/1024-7.5-308.23)/5376.6*16}'
wc -c < models/Meta-Llama-3.1-8B-Instruct-Q2_K.gguf  | awk '{printf "%.2f bpw\n", ($1/1024/1024-7.5-410.98*2)/13313.0*16}'
```

8B | ppl | +- | bpw
--- | --- | --- | ---
BF16**  | 7.3221 | 0.04673 | 16
Q8_0**  | 7.3311 | 0.04679 | 8.50
Q4_0*   | 7.6185 | 7.7364 | 4.50
Q4_K_M* | 7.5071 | 7.5322 | 4.80
IQ4_XS  | 7.5242 | 0.04821 | 4.27
IQ4_KS  | 7.4781 | 0.04777 | 4.28
IQ4_KSS | 7.5934 | 0.04848 | 4.04
IQ3_XXS | 8.4682 | 0.05315 | 3.07
IQ3_KT  | 8.1155 | 0.05161 | 3.18
exl3-3.0bpw | 7.7937 | 0.04986 | 3.01
| | |
IQ2_KL | 9.3848 | 0.06148 | 2.72
IQ2_K | 10.1837 | 0.06689 | 2.50
IQ2_M | 9.5655 | 0.06230 | 2.70
IQ2_S | 10.5209 | 0.06981 | 2.48
IQ2_KL+IQ2_KS | 7.5117 | 
IQ2_K+IQ2_KS | 7.5172 | 7.5225
IQ2_M+IQ2_KS | 7.4951 | 0.04796
IQ2_S+IQ2_KS | 7.5634 | 7.5791
| | |
exl3-2.0bpw | 10.1817 | 0.06631 | 2.01
exl3-2.25bpw | 9.3610 | 0.06127 | 2.26
exl3-2.5bpw | 8.6902 | 0.05687 | 2.51
exl3-2.0bpw+IQ2_KS | 7.5922 | 0.04846
exl3-2.25bpw+IQ2_KS | 7.5636 | 0.04823
exl3-2.5bpw+IQ2_KS | 7.4845 | 0.04792

</details>

<details>
<summary>perplexity results (legacy)</summary>

model | ppl | +- | bpw/size
--- | --- | --- | ---
[gptq-g32](https://huggingface.co/ModelCloud/Llama-3.2-1B-Instruct-gptqmodel-4bit-vortex-v2.5?show_file_info=model.safetensors) | 14.8567 | 0.11085 | 4@weight+(16@scale+4@zero)/32+32@g_dx/out_dim
[gptq-g128](https://hf-mirror.com/shuyuej/Llama-3.2-1B-Instruct-GPTQ?show_file_info=model.safetensors) | 17.8053 | 0.13670 | 4@weight+(16@scale+4@zero)/128+32@g_dx/out_dim
[awq](https://huggingface.co/AMead10/Llama-3.2-1B-Instruct-AWQ?show_file_info=model.safetensors)  | 15.2694 | 0.11399 | 4@weight+(16@scale+4@zero)/128
[IQ4_XS](https://hf-mirror.com/bartowski/Llama-3.2-1B-Instruct-GGUF?show_file_info=Llama-3.2-1B-Instruct-IQ4_XS.gguf) | 14.6856 | 0.10988 | 4.25 / 709MB
[Q4_0](https://hf-mirror.com/bartowski/Llama-3.2-1B-Instruct-GGUF?show_file_info=Llama-3.2-1B-Instruct-Q4_0.gguf) | 15.3289 | 0.11394 | 738MB
[Q4_K_S](https://hf-mirror.com/bartowski/Llama-3.2-1B-Instruct-GGUF?show_file_info=Llama-3.2-1B-Instruct-Q4_K_S.gguf) | 14.5792 | 0.10848 | 740MB
[Q5_K_S](https://hf-mirror.com/bartowski/Llama-3.2-1B-Instruct-GGUF?show_file_info=Llama-3.2-1B-Instruct-Q5_K_S.gguf) | 14.0893 | 0.10462 | 852MB
[Q5_K_M](https://hf-mirror.com/bartowski/Llama-3.2-1B-Instruct-GGUF?show_file_info=Llama-3.2-1B-Instruct-Q5_K_.gguf) | 14.0558 | 0.10458 | 870MB
llmc-awq-omniq | 14.0102 | 0.10419 | 4@weight+16@scale/128?
[w4a16g128asym](https://hf-mirror.com/numen-tech/Llama-3.2-1B-Instruct-w4a16g128asym) | 19.5997 | 0.14797 | 4@weight+16@scale/128
    
</details>



In [ ]:
# align with https://github.com/ikawrakow/ik_llama.cpp/discussions/63
!ik_llama.cpp/cuda_build/bin/llama-perplexity -m {model} -f wikitext-2-raw/wiki.test.raw -t 1 -ngl 100 #--chunks 1

In [ ]:
!llama.cpp/cuda_build/bin/llama-bench -m {model} -p 512 -n 128 -t 1 -ngl 100

In [ ]:
!llama.cpp/cpu_build/bin/llama-bench -m {model} -p 4,8,32,128,512 -n 128 -t 8

## exllamav3

In [ ]:
# !git clone https://github.com/turboderp-org/exllamav3 --depth 1
assert os.path.exists(exllamav3)
%cd exllamav3
!pip install -r requirements.txt
!pip install .
%cd ..

In [ ]:
%cd exllamav3
bpw = 2.0
quant_type = f"exl3-{str(bpw)}bpw"
model = f"models/{model_name}-{quant_type}"
!python convert.py -i {"../"+hf_model} -o {"../"+model} -w f"/tmp/{model_name}-{quant_type}" -b {str(bpw)}
%cd -

## dump information

In [ ]:
from safetensors import safe_open
model_path = f"{model}/model.safetensors"

with safe_open(model_path, framework="pt") as f:
    tensor_names = f.keys()
    byte_cnt = 0
    for name in tensor_names:
        if "model.layers" in name:
            tensor = f.get_tensor(name)
            byte_cnt += 2*tensor.numel()
byte_cnt/1024.0/1024.0

In [ ]:
from safetensors import safe_open
model_path = f"{hf_model}/model.safetensors"

with safe_open(model_path, framework="pt") as f:
    tensor_names = f.keys()
    for name in tensor_names:
        tensor = f.get_tensor(name)
        print(f"name: {name}")
        print(f"shape: {tensor.shape}")
        print(f"type: {tensor.dtype}")
        print("-" * 50)

In [ ]:
with safe_open(model_path, framework="pt") as f:
  # tensor = f.get_tensor("model.norm.weight") # 'output_norm.weight'
  tensor = f.get_tensor("model.layers.0.input_layernorm.weight")
tensor

# Consider residual

In [ ]:
draft_type = "IQ2_K"
# draft_type = "IQ2_KL"
# draft_type = "IQ2_S"
# draft_type = "IQ2_M"
# draft_type = "exl3-2.0bpw"
# draft_type = "exl3-2.25bpw"
# draft_type = "exl3-2.5bpw"
draft_model_prefix = f"models/{model_name}-{draft_type}"

In [ ]:
# Given residual weight corresponding to draft_type to quant it
imatrix_file = f"--imatrix models/{model_name}.imatrix"
quant_option = "--token-embedding-type q6_K"
residual_model = draft_model_prefix + "-residual.gguf"
residual_qmodel = draft_model_prefix + "-residual-IQ2_KS.gguf"
# we don't care output-tensor and token-embedding, just don't want to quant them exactly
!ik_llama.cpp/cuda_build/bin/llama-quantize {imatrix_file} --pure {quant_option} {residual_model} {residual_qmodel} IQ2_KS
residual_model

<details>
<summary>imatrix modification for dumping activations</summary>

```diff
diff --git a/examples/imatrix/imatrix.cpp b/examples/imatrix/imatrix.cpp
index 2e03a4a0..2f10ac30 100644
--- a/examples/imatrix/imatrix.cpp
+++ b/examples/imatrix/imatrix.cpp
@@ -39,6 +39,7 @@ static void print_usage(int argc, char ** argv, const gpt_params & params) {
 
 struct Stats {
     std::vector<float> values;
+    std::vector<float> activations;
     std::vector<int> counts;
     int ncall = 0;
     int n_as = 1;
@@ -341,6 +342,7 @@ bool IMatrixCollector::collect_imatrix(struct ggml_tensor * t, bool ask, void *
             // Hence, the storage we need is src0->ne[0]*src0->ne[2].
             e.values.resize(src0->ne[0]*src0->ne[2], 0);
             e.counts.resize(src0->ne[0]*src0->ne[2], 0);
+            e.activations.resize(src1->ne[0]*src1->ne[1]);
         }
         else if (e.values.size() != (size_t)(src0->ne[0]*src0->ne[2])) {
             fprintf(stderr, "Oops: inconsistent size for %s (%d vs %d)\n", wname.c_str(), (int)e.values.size(), (int)src1->ne[0]);
@@ -357,8 +359,10 @@ bool IMatrixCollector::collect_imatrix(struct ggml_tensor * t, bool ask, void *
             auto counts = e.counts.data() + i02*src0->ne[0];
             for (int i11 = 0; i11 < (int)src1->ne[1]; ++i11) {
                 const float * x = (const float *)((const char *)data + i11*src1->nb[1] + i12*src1->nb[2]);
+                auto activations = e.activations.data() + i11*src1->ne[0];
                 for (int j = 0; j < (int)src1->ne[0]; ++j) {
                     values[j] += x[j]*x[j];
+                    activations[j] = x[j]*x[j];
                     counts[j]++;
                     if (!std::isfinite(values[j])) {
                         fprintf(stderr, "%f detected in %s\n", e.values[j], wname.c_str());
@@ -482,6 +486,9 @@ void IMatrixCollector::save_imatrix(int ncall) const {
                 tmp[i] = (stat.values[i] / static_cast<float>(stat.counts[i])) * static_cast<float>(stat.ncall);
             }
             out.write((const char*)tmp.data(), nval*sizeof(float));
+            int nact = stat.activations.size();
+            out.write((const char *) &nact, sizeof(nact));
+            out.write((const char*)stat.activations.data(), nact*sizeof(float));
         }
     }
 

```

</details>

In [ ]:
target_model = draft_model_prefix + "-IQ2_KS.gguf"
imatrix_file = draft_model_prefix + "-IQ2_KS.imatrix"

calib_file = f"calibration_data/calibration_datav3.txt"
# modified ik_llama.cpp imatrix to dump activations
# Attention: the corresponding load_gguf_imatrix func in playgguf.ipynb requires 32-bit int range,
# therefore, the dumped activations imatrix file should smaller than 2GB
# therefore, the batch size should be tried to fit the above constraints
!ik_llama.cpp/cuda_build/bin/llama-imatrix -m {target_model} -f {calib_file} --output-file {imatrix_file} -ngl 100 -b 256 -c 256 --chunks 32 #-t 1
target_model


<details>
<summary>1B, but we don't care so much</summary>

1B | #Q1 | #Q2 | #Q3 | #Q4 |
--- | --- | --- |--- | ---
IQ2_KL+IQ2_KS | 60.307% | 48.333% | 61.364% | 75.000%
IQ2_K+IQ2_KS | 52.560% | 39.919% | 57.558% | 74.242%
IQ2_M+IQ2_KS | 66.786% | 43.929% | 66.026% | 62.500%
IQ2_S+IQ2_KS | 52.841% | 34.375% | 52.632% | 57.895%
exl3-2.5bpw+IQ2_KS | 62.597% | 50.000% | 69.952% | 77.941%
exl3-2.25bpw+IQ2_KS | 56.818% | 48.120% | 65.278% | 61.111%
exl3-2.0bpw+IQ2_KS | 49.144% | 51.230% | 85.991%  | 69.444%

</details>

> self speculative

8B | #Q1 | #Q2 | #Q3 | #Q4 |
--- | --- | --- |--- | ---
IQ2_M+IQ2_KS | 73.288% | 73.881% | 72.917% | 92.593%
IQ2_S+IQ2_KS | 69.505% | 58.562% | 76.339% | 92.593%
IQ2_KL+IQ2_KS | 
IQ2_K+IQ2_KS | 68.000% | 70.902% | 80.882% | 75.000%
exl3-2.5bpw+IQ2_KS | 74.671% | 74.643% | 52.679% | 75.000%
exl3-2.25bpw+IQ2_KS | 73.739% | 61.397% | 66.667% | 79.688%
exl3-2.0bpw+IQ2_KS | 53.313% | 55.340% | 65.789% | 79.167%

> orignal

8B | #Q1 | #Q2 | #Q3 | #Q4 |
--- | --- | --- |--- | ---
IQ2_M+IQ2_KS | 77.632%| 71.121% | 85.714% | 70.968%
IQ2_S+IQ2_KS | 76.087% | 61.594% | 60.473% | 82.500%
IQ2_KL+IQ2_KS | 74.786% | 80.500% | 72.093% | 91.667%
IQ2_K+IQ2_KS | 63.261% | 62.805% | 63.362% | 78.333%
exl3-2.5bpw+IQ2_KS | 74.671% | 74.643% | 52.679% | 75.000%
exl3-2.25bpw+IQ2_KS | 73.739% | 61.397% | 66.667% | 79.688%
exl3-2.0bpw+IQ2_KS | 53.313% | 55.340% | 65.789% | 79.167%


In [ ]:
# target_model = f"models/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf"
# draft_model  = f"models/Llama-3.2-1B-Instruct-Q8_0.gguf"

target_model = draft_model_prefix + ".gguf"
draft_model  = draft_model_prefix + "-residual-IQ2_KS.gguf"

prefix = "<|start_header_id|>user<|end_header_id|>\n\n"
suffix = "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

prompt = prefix+"Write 20 sentences about summer."+suffix #Q1
# prompt = prefix+"Who was the first prime minister of Britain?"+suffix #Q2
# prompt = prefix+"How many persons are needed to power a 800W toaster?"+suffix #Q3
# prompt = prefix+"Write the Quicksort algorithm in TypeScript."+suffix #Q4

# https://github.com/ggml-org/llama.cpp/discussions/10466#discussioncomment-11501175
!llama.cpp/cpu_build/bin/llama-speculative -m {target_model} -md {draft_model} \
-p "{prompt}" -c 2048 -n 512 --temp 0 --top-k 1 --seed 42 --draft-max 4 --draft-min 0 --draft-p-min 0.0 -t 8
draft_model

In [ ]:
!llama.cpp/cuda_build/bin/llama-cli -m {target_model} -p "{prompt}" -c 2048 -n 512 --temp 0 --top-k 1 --seed 42 -ngl 100 -no-cnv 2>/dev/null
target_model

In [ ]:
!llama.cpp/cuda_build/bin/llama-cli -m {draft_model}  -p "{prompt}" -c 2048 -n 512 --temp 0 --top-k 1 --seed 42 -ngl 100 -no-cnv 2>/dev/null
draft_model